## Mimic EDA
#### Predicting and Understanding Unexpected Respiratory Decompensation in Critical Care Using Sparse and Heterogeneous Clinical Data

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import os 
import random
import datetime
from sqlalchemy import create_engine

random.seed(22891)

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

engine = create_engine("postgresql+psycopg2://{}:{}@/{}".format(sqluser, sqluser, dbname))

In [2]:
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)

In [3]:
from mimic_queries import services_query, demo_query, vent_query, vital_and_labs_query
from mimic_queries import clean_demo_df, comob_query

### Mimic Services

| Service |                                 Description                                               |
|---------|-------------------------------------------------------------------------------------------|
|   CMED  | Cardiac Medical - for non-surgical cardiac related admissions                             |
|   CSURG |	Cardiac Surgery - for surgical cardiac admissions                                         |
|   DENT  |	Dental - for dental/jaw related admissions                                                |
|   ENT   |	Ear, nose, and throat - conditions primarily affecting these areas                        |
|   GU	  | Genitourinary - reproductive organs/urinary system                                        |
|   GYN	  | Gynecological - female reproductive systems and breasts                                   |
|   MED	  | Medical - general service for internal medicine                                           |
|   NB	  | Newborn - infants born at the hospital                                                    |
|   NBB	  | Newborn baby - infants born at the hospital                                               |
|   NMED  |	Neurologic Medical - non-surgical, relating to the brain                                  |
|   NSURG |	Neurologic Surgical - surgical, relating to the brain                                     |
|   OBS	  | Obstetrics - conerned with childbirth and the care of women giving birth                  |
|   ORTHO |	Orthopaedic - surgical, relating to the musculoskeletal system                            |
|   OMED  |	Orthopaedic medicine - non-surgical, relating to musculoskeletal system                   |
|   PSURG |	Plastic - restortation/reconstruction of the human body (including cosmetic or aesthetic) |
|   PSYCH |	Psychiatric - mental disorders relating to mood, behaviour, cognition, or perceptions     |
|   SURG  |	Surgical - general surgical service not classified elsewhere                              |
|   TRAUM |	Trauma - injury or damage caused by physical harm from an external source                 |
|   TSURG |	Thoracic Surgical - surgery on the thorax, located between the neck and the abdomen       |
|   VSURG |	Vascular Surgical - surgery relating to the circulatory system                            |

In [4]:
df_services = pd.read_sql(services_query(), conn)

In [5]:
df_services.head(3)

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,471,135879,2122-07-22 14:07:27,TSURG,MED
1,471,135879,2122-07-26 18:31:49,MED,TSURG
2,472,173064,2172-09-28 19:22:15,None,CMED


In [6]:
df_services.curr_service.value_counts()

MED      24866
CMED      9135
CSURG     7911
NB        7806
SURG      5514
NSURG     4100
TRAUM     3137
NMED      2957
OMED      2286
VSURG     1455
TSURG     1338
ORTHO     1101
GU         436
NBB        346
GYN        306
PSURG      283
ENT        247
OBS        113
DENT         5
PSYCH        1
Name: curr_service, dtype: int64

### Mimic Demographics

In [7]:
df_demo = pd.read_sql_query(demo_query(), conn)

In [8]:
df_demo_clean = clean_demo_df(df_demo, 27)  #  stay in ICU a minimum of 27h

In [9]:
len(df_demo), len(df_demo_clean)

(61051, 43324)

In [10]:
df_demo_clean.adult_icu.value_counts()

1    39553
0     3771
Name: adult_icu, dtype: int64

In [ ]:
df_demo_clean.age.value_counts()

In [ ]:
df_demo_clean.admission_type.value_counts()

In [ ]:
df_demo_clean.ethnicity.value_counts()

In [ ]:
df_demo_clean.diagnosis.value_counts()

In [ ]:
df_demo_clean.first_hosp_stay.value_counts()

In [ ]:
df_demo_clean.first_icu_stay.value_counts()

In [ ]:
df_demo_clean.mort_icu.value_counts()

### Mimic Mechanical Ventilation

In [11]:
df_vent = pd.read_sql_query(vent_query(), conn)
# there are 34 instances of MV without an icustay_id, we remove those
# also, there are a few indididuals with multiple instances of MV that we remove
df_vent = df_vent[~df_vent.icustay_id.isnull()]
df_vent = df_vent.astype({"icustay_id": int})
df_vent = df_vent[df_vent['ventnum'] == 1]

In [12]:
len(df_vent), len(df_demo_clean)

(26800, 43324)

In [13]:
df_cohort = df_demo_clean.merge(df_vent, left_on='icustay_id', right_on='icustay_id', how='left')
df_cohort = df_cohort.merge(df_services, 
                            left_on=['subject_id','hadm_id'], 
                            right_on=['subject_id','hadm_id'], how='inner')


df_cohort = df_cohort[(df_cohort.transfertime > df_cohort.admittime) & 
                      (df_cohort.transfertime < df_cohort.outtime)]

df_cohort = df_cohort[(np.isnan(df_cohort.ventnum)) | 
                      (df_cohort.mv_start > df_cohort.transfertime)]

In [14]:
len(df_demo_clean), len(df_cohort)

(43324, 42609)

In [15]:
df_demo_clean.subject_id.nunique(), df_cohort.subject_id.nunique()

(33472, 27753)

#### Almost exactly following:
Cohort: We used data from adult patients from MIMIC
III in this study (age greater than 16 at admission). Next,
for patients with multiple hospital or ICU stays, we only
considered the first ICU stay for the first hospital stay. We
excluded patients admitted under surgical service because
surgical patients are frequently intubated due to anesthesia
rather than respiratory failure. Patients who were intubated on
admission were excluded, as were patients who were intubated
or discharged from the ICU within 27 hours from admission.
Finally, we removed patients who requested a withdrawal of
care within the first 27 hours as these patients will not be
intubated despite respiratory decompensation

In [16]:
# keep only adult patient admissions
df_cohort = df_cohort.loc[df_cohort.adult_icu == 1]
# keep only admissions that are first time ICU admissions and hospitalizations
df_cohort = df_cohort.loc[(df_cohort.first_icu_stay == 1) & (df_cohort.first_hosp_stay == 1)]
# remove admissions of patients intubated at admition time
df_cohort = df_cohort[np.isnan(df_cohort.ventnum) | (df_cohort.mv_start - df_cohort.admittime).dt.days > 0]
# remove admissions of surgical patients 
surgical = ['CSURG', 'NSURG', 'ORTHO', 'SURG', 'TSURG', 'VSURG']
df_cohort = df_cohort.loc[~np.isin(df_cohort.curr_service, surgical)]

In [17]:
len(df_cohort), df_cohort.subject_id.nunique()

(13920, 12791)

In [18]:
cols = list(set(df_cohort.columns) - {'transfertime', 'prev_service', 'curr_service'})
df_cohort = df_cohort.sort_values(['transfertime']).drop_duplicates(cols, keep='last')

In [19]:
len(df_cohort)

12791

In [20]:
df_cohort.ventnum.value_counts(dropna=False)

NaN    9541
1.0    3250
Name: ventnum, dtype: int64

In [22]:
conn.execute("DROP TABLE IF EXISTS mv_users;")

In [23]:
conn.close()
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)
df_columns = ['subject_id', 'hadm_id', 'icustay_id', 'ventnum', 'mv_start', 'transfertime']
df_cohort[df_columns].to_sql('mv_users', conn, if_exists='replace', index=False)

In [24]:
conn.close()
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)

In [25]:
df_comob = pd.read_sql(comob_query(), conn)

In [26]:
df_comob.head(1)

,hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,100001,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
df_comob.columns

Index(['hadm_id', 'congestive_heart_failure', 'cardiac_arrhythmias',
       'valvular_disease', 'pulmonary_circulation', 'peripheral_vascular',
       'hypertension', 'paralysis', 'other_neurological', 'chronic_pulmonary',
       'diabetes_uncomplicated', 'diabetes_complicated', 'hypothyroidism',
       'renal_failure', 'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression'],
      dtype='object')

In [31]:
len(df_comob)

58976

#### Loosely following:
Data extraction: For each patient in our cohort, we
extracted data from a window of size W hours, located L
hours before a given event time. Specifically, we extracted
a number of features, (see Table I), from the data window
[te − W − L, te − L], where te represents the event time. For
intubated patients, the event time was the time of intubation.
For non-intubated patients, the event time was a random time
after the 27th hour during their ICU stay. For non-intubated
patients with a code status change (e.g. changed to do not
resuscitate or do not intubate), we ensured that the event
time was before the code status change. For all patients,
we evaluated and extracted features from data windows with
window sizes of W ∈ {8, 12, 16, 20, 24} hours and lead times
of L ∈ {3, 6, 9, 12, 15} hours.

In [27]:
vitals_labs_df = pd.read_sql(vital_and_labs_query(27, 3), conn)

In [28]:
len(vitals_labs_df), vitals_labs_df.subject_id.nunique()

(204198, 10415)

In [29]:
vitals_labs_df.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'timepoint', 'event_time',
       'ventilated', 'mv_start', 'aniongap', 'albumin', 'bicarbonate',
       'bilirubin', 'creatinine', 'chloride', 'glucose', 'hematocrit',
       'hemoglobin', 'lactate', 'magnesium', 'phosphate', 'platelet',
       'potassium', 'ptt', 'inr', 'pt', 'sodium', 'bun', 'wbc',
       'heartrate_mean', 'sysbp_mean', 'diasbp_mean', 'meanbp_mean',
       'resprate_mean', 'tempc_mean', 'spo2_mean', 'glucose_mean'],
      dtype='object')

In [38]:
cols = list(set(df_cohort.columns) - {'mv_start'})
df_selected = vitals_labs_df.merge(df_cohort[cols], 
                                   left_on=['subject_id', 'hadm_id', 'icustay_id'],
                                   right_on=['subject_id', 'hadm_id', 'icustay_id'])
df_selected = df_selected.merge(df_comob, left_on='hadm_id', right_on='hadm_id', how='left')

In [39]:
len(df_selected), df_selected.subject_id.nunique()

(204198, 10415)

In [40]:
from IPython.display import display
pd.options.display.max_columns = None
display(df_selected.head())

,subject_id,hadm_id,icustay_id,timepoint,event_time,ventilated,mv_start,aniongap,albumin,bicarbonate,bilirubin,creatinine,chloride,glucose,hematocrit,hemoglobin,lactate,magnesium,phosphate,platelet,potassium,ptt,inr,pt,sodium,bun,wbc,heartrate_mean,sysbp_mean,diasbp_mean,meanbp_mean,resprate_mean,tempc_mean,spo2_mean,glucose_mean,age,first_icu_stay,adult_icu,first_careunit,diagnosis,curr_service,dischtime,admission_type,mort_icu,gender,admittime,los_icu,mv_hours,los_icu_hr,mv_end,los_hospital,first_hosp_stay,outtime,intime,ventnum,prev_service,transfertime,ethnicity,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,diabetes_uncomplicated,diabetes_complicated,hypothyroidism,renal_failure,liver_disease,peptic_ulcer,aids,lymphoma,metastatic_cancer,solid_tumor,rheumatoid_arthritis,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
0,4,185777,294638,3,2191-03-17 03:29:31,0,NaT,17.0,2.8,24.0,2.2,0.5,97.0,140.0,34.2,11.5,2.1,1.9,3.2,207.0,3.1,31.3,1.0,12.3,135.0,9.0,9.7,97.0,119.0,69.0,85.666702,28.0,NaN,98.0,NaN,47.0,1,1,MICU,"FEVER,DEHYDRATION,FAILURE TO THRIVE",MED,2191-03-23 18:41:00,EMERGENCY,0,0,2191-03-16 00:28:00,1.0,NaN,40.0,NaT,7.0,1,2191-03-17 16:46:31,2191-03-16 00:29:31,NaN,None,2191-03-16 00:29:31,white,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,185777,294638,4,2191-03-17 03:29:31,0,NaT,17.0,2.8,24.0,2.2,0.5,97.0,140.0,34.2,11.5,2.1,1.9,3.2,207.0,3.1,31.3,1.0,12.3,135.0,9.0,9.7,94.0,NaN,NaN,NaN,NaN,NaN,97.0,153.0,47.0,1,1,MICU,"FEVER,DEHYDRATION,FAILURE TO THRIVE",MED,2191-03-23 18:41:00,EMERGENCY,0,0,2191-03-16 00:28:00,1.0,NaN,40.0,NaT,7.0,1,2191-03-17 16:46:31,2191-03-16 00:29:31,NaN,None,2191-03-16 00:29:31,white,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,185777,294638,5,2191-03-17 03:29:31,0,NaT,17.0,2.8,24.0,2.2,0.5,97.0,140.0,34.2,11.5,2.1,1.9,3.2,207.0,3.1,31.3,1.0,12.3,135.0,9.0,9.7,99.0,133.0,79.0,97.000000,26.0,NaN,98.0,NaN,47.0,1,1,MICU,"FEVER,DEHYDRATION,FAILURE TO THRIVE",MED,2191-03-23 18:41:00,EMERGENCY,0,0,2191-03-16 00:28:00,1.0,NaN,40.0,NaT,7.0,1,2191-03-17 16:46:31,2191-03-16 00:29:31,NaN,None,2191-03-16 00:29:31,white,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,185777,294638,6,2191-03-17 03:29:31,0,NaT,17.0,2.8,24.0,2.2,0.5,97.0,140.0,34.2,11.5,2.1,1.9,3.2,207.0,3.1,31.3,1.0,12.3,135.0,9.0,9.7,92.0,NaN,NaN,NaN,24.0,36.666667,97.0,NaN,47.0,1,1,MICU,"FEVER,DEHYDRATION,FAILURE TO THRIVE",MED,2191-03-23 18:41:00,EMERGENCY,0,0,2191-03-16 00:28:00,1.0,NaN,40.0,NaT,7.0,1,2191-03-17 16:46:31,2191-03-16 00:29:31,NaN,None,2191-03-16 00:29:31,white,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,185777,294638,7,2191-03-17 03:29:31,0,NaT,17.0,2.8,24.0,2.2,0.5,97.0,140.0,34.2,11.5,2.1,1.9,3.2,207.0,3.1,31.3,1.0,12.3,135.0,9.0,9.7,89.0,139.0,81.0,100.333000,25.0,NaN,97.0,NaN,47.0,1,1,MICU,"FEVER,DEHYDRATION,FAILURE TO THRIVE",MED,2191-03-23 18:41:00,EMERGENCY,0,0,2191-03-16 00:28:00,1.0,NaN,40.0,NaT,7.0,1,2191-03-17 16:46:31,2191-03-16 00:29:31,NaN,None,2191-03-16 00:29:31,white,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
df_selected.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'timepoint', 'event_time',
       'ventilated', 'mv_start', 'aniongap', 'albumin', 'bicarbonate',
       'bilirubin', 'creatinine', 'chloride', 'glucose', 'hematocrit',
       'hemoglobin', 'lactate', 'magnesium', 'phosphate', 'platelet',
       'potassium', 'ptt', 'inr', 'pt', 'sodium', 'bun', 'wbc',
       'heartrate_mean', 'sysbp_mean', 'diasbp_mean', 'meanbp_mean',
       'resprate_mean', 'tempc_mean', 'spo2_mean', 'glucose_mean', 'age',
       'first_icu_stay', 'adult_icu', 'first_careunit', 'diagnosis',
       'curr_service', 'dischtime', 'admission_type', 'mort_icu', 'gender',
       'admittime', 'los_icu', 'mv_hours', 'los_icu_hr', 'mv_end',
       'los_hospital', 'first_hosp_stay', 'outtime', 'intime', 'ventnum',
       'prev_service', 'transfertime', 'ethnicity', 'congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       '

In [42]:
conn.execute("DROP TABLE IF EXISTS mimic_users_study;")
conn.close()

In [43]:
schema_name = 'mimiciii'
conn = engine.connect()
conn.execute('SET search_path to ' + schema_name)
df_selected.to_sql('mimic_users_study', conn, if_exists='replace', index=False)

In [44]:
conn.close()